# Stable-Diffusion-Comfyui Kaggle 云端部署 (完全免费，无需显卡)
**venv 版本：torch: 2.0.1+cu118  •  xformers: 0.0.20**
<!--- 
这里是作者的署名和群号码，未经同意请勿删除，删一句家里人少一个
--->
<!--- 
倒卖狗全家死光
--->


# 仅用于学术交流使用，禁止将源代码用于出售
# 有问题请加群 632428790
# 没来得及优化，姑且用着吧（，随时可能更新
<!--- 
这里是作者的署名和群号码，未经同意请勿删除，删一句家里人少一个
--->
<!--- 
倒卖狗全家死光
--->


-----

In [1]:
from pathlib import Path
import subprocess
import pandas as pd
import shutil
import os
import time
import re
import gc
import requests
import zipfile
import threading
import time
import socket
import concurrent.futures
from concurrent.futures import ProcessPoolExecutor

In [2]:
#@title Environment Setup
from pathlib import Path

OPTIONS = {}

UPDATE_COMFY_UI = True  #@param {type:"boolean"}
WORKSPACE = '/kaggle/working/ComfyUI'
OPTIONS['UPDATE_COMFY_UI'] = UPDATE_COMFY_UI
def comfyui():
    print('-= 线程1：安装ComfyUI -=')
    time.sleep(0.11)
    %cd /kaggle/working/
    ![ ! -d $WORKSPACE ] && echo -= ready to git colne ComfyUI from QQ2575044704 =- && git clone https://github.com/2575044704/ComfyUI
    %cd $WORKSPACE

    if OPTIONS['UPDATE_COMFY_UI']:
      !echo -= Updating ComfyUI =-
      !git pull


def venv_install():
    time.sleep(0.05)
    print('-= 线程2：安装apt-get和Python环境 -=')
    !apt-get update & npm install -g localtunnel
    !apt-get -y install aria2
    !echo -= Install dependencies =-
    %cd /opt/conda/envs
    if os.path.exists('venv'):
        print('环境已安装')
    else:
        %cd /kaggle/working/
        if not os.path.exists('venv.tar.gz'):
            print('下载 VENV')
            !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/sukaka/venv_ai_drow/resolve/main/sd_webui/sd_webui_torch201_cu118_xf20.tar.gz -o venv.tar.gz
        print('successfully downloaded venv.tar.gz')
        %cd /opt/conda/envs/
        !mkdir venv
        %cd venv
        print('安装 venv')
        os.system('apt -y install -qq pigz > /dev/null 2>&1')
        !pigz -dc -p 10 /kaggle/working/venv.tar.gz | tar xf -
        !source /opt/conda/bin/activate venv
        print('环境安装完毕')



In [3]:
#功能函数，内存优化
def libtcmalloc():
    if os.path.exists('/kaggle/temp'):
        os.chdir('/kaggle')
        os.chdir('temp')
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        print('内存优化已安装')
    else:
        
        #if use_frpc:
            #!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/datasets/ACCA225/Frp/resolve/main/frpc -d /kaggle/working/frpc -o frpc
        os.system('pip install -q pyngrok ')
        os.chdir('/kaggle')
        os.makedirs('temp', exist_ok=True)
        os.chdir('temp')
        os.system('wget -qq  http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb')
        os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb')
        os.system('wget -qq  https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb')
        os.system('apt install -qq libunwind8-dev -y')
        !dpkg -i *.deb
        os.environ["LD_PRELOAD"] = "libtcmalloc.so"
        !rm *.deb
        print('内存优化已安装')

In [4]:
# Checkpoints

# SD1
# !wget -c https://huggingface.co/runwayml/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt -P ./models/checkpoints/
def model():
    time.sleep(0.1)
    print('-= 线程3：30秒后自动下载模型 -=')
    time.sleep(30)
    print('正在下载sdxl 1.0基本模型,大小6.9G')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/stabilityai/stable-diffusion-xl-base-1.0/resolve/main/sd_xl_base_1.0.safetensors"  -d /kaggle/working/ComfyUI/models/checkpoints/ -o sd_xl_base_1.0.safetensors
    print('正在下载sdxl 1.0优化模型,大小6.5G')
    !aria2c --console-log-level=error -c -x 16 -s 16 -k 1M "https://huggingface.co/stabilityai/stable-diffusion-xl-refiner-1.0/resolve/main/sd_xl_refiner_1.0.safetensors"  -d /kaggle/working/ComfyUI/models/checkpoints/ -o sd_xl_refiner_1.0.safetensors
    print('下载完成')
# SD2
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/stabilityai/stable-diffusion-2-1/resolve/main/v2-1_768-ema-pruned.safetensors -P ./models/checkpoints/

# Some SD1.5 anime style
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix2/AbyssOrangeMix2_hard.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A1_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/Models/AbyssOrangeMix3/AOM3A3_orangemixs.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/Linaqruf/anything-v3.0/resolve/main/anything-v3-fp16-pruned.safetensors -P ./models/checkpoints/

# Waifu Diffusion 1.5 (anime style SD2.x 768-v)
#!wget -c https://huggingface.co/waifu-diffusion/wd-1-5-beta2/resolve/main/checkpoints/wd-1-5-beta2-fp16.safetensors -P ./models/checkpoints/


# unCLIP models
#!wget -c https://huggingface.co/comfyanonymous/illuminatiDiffusionV1_v11_unCLIP/resolve/main/illuminatiDiffusionV1_v11-unclip-h-fp16.safetensors -P ./models/checkpoints/
#!wget -c https://huggingface.co/comfyanonymous/wd-1.5-beta2_unCLIP/resolve/main/wd-1-5-beta2-aesthetic-unclip-h-fp16.safetensors -P ./models/checkpoints/


# VAE
# !wget -c https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors -P ./models/vae/
#!wget -c https://huggingface.co/WarriorMama777/OrangeMixs/resolve/main/VAEs/orangemix.vae.pt -P ./models/vae/
#!wget -c https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt -P ./models/vae/


# Loras
#!wget -c https://civitai.com/api/download/models/10350 -O ./models/loras/theovercomer8sContrastFix_sd21768.safetensors #theovercomer8sContrastFix SD2.x 768-v
#!wget -c https://civitai.com/api/download/models/10638 -O ./models/loras/theovercomer8sContrastFix_sd15.safetensors #theovercomer8sContrastFix SD1.x


# T2I-Adapter
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_depth_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_seg_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_sketch_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_keypose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_openpose_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_color_sd14v1.pth -P ./models/controlnet/
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_canny_sd14v1.pth -P ./models/controlnet/

# T2I Styles Model
#!wget -c https://huggingface.co/TencentARC/T2I-Adapter/resolve/main/models/t2iadapter_style_sd14v1.pth -P ./models/style_models/

# CLIPVision model (needed for styles model)
#!wget -c https://huggingface.co/openai/clip-vit-large-patch14/resolve/main/pytorch_model.bin -O ./models/clip_vision/clip_vit14.bin


# ControlNet
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_ip2p_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11e_sd15_shuffle_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_canny_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11f1p_sd15_depth_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_inpaint_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_lineart_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_mlsd_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_normalbae_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_openpose_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_scribble_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_seg_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15_softedge_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11p_sd15s2_lineart_anime_fp16.safetensors -P ./models/controlnet/
#!wget -c https://huggingface.co/comfyanonymous/ControlNet-v1-1_fp16_safetensors/resolve/main/control_v11u_sd15_tile_fp16.safetensors -P ./models/controlnet/


# Controlnet Preprocessor nodes by Fannovel16
#!cd custom_nodes && git clone https://github.com/Fannovel16/comfy_controlnet_preprocessors; cd comfy_controlnet_preprocessors && python install.py


# GLIGEN
#!wget -c https://huggingface.co/comfyanonymous/GLIGEN_pruned_safetensors/resolve/main/gligen_sd14_textbox_pruned_fp16.safetensors -P ./models/gligen/


# ESRGAN upscale model
#!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x2.pth -P ./models/upscale_models/
#!wget -c https://huggingface.co/sberbank-ai/Real-ESRGAN/resolve/main/RealESRGAN_x4.pth -P ./models/upscale_models/



In [5]:
ngrok = False
def ngrok():
    !pip install pyngrok
    %cd /kaggle/working/hello-ngrok
    from pyngrok import conf, ngrok
    ngrokToken = '填写ngrok的token'
    conf.get_default().auth_token = ngrokToken
    conf.get_default().monitor_thread = False
    ssh_tunnels = ngrok.get_tunnels(conf.get_default())
    if len(ssh_tunnels) == 0:
        ssh_tunnel = ngrok.connect(8188)
        print('address：'+ssh_tunnel.public_url)
    else:
        print('address：'+ssh_tunnels[0].public_url)
    


import subprocess
import threading
import time
import socket
import urllib.request

def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()
        print("\nComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)\n")
        print("\033[91m请通过公网IP进入ComfyUI，你的公网IP地址是:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"), "\033[0m")
        p = subprocess.Popen(["lt", "--port", "{}".format(port)], stdout=subprocess.PIPE)
        for line in p.stdout:
            print(line.decode(), end='')

def start():
    threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
    %cd
    %cd /kaggle/working/ComfyUI
    !/opt/conda/envs/venv/bin/python3 main.py

# 执行区域，输出结果在下方看

In [6]:
'''
执行函数
'''
!echo '采用多线程启动，加快启动速度'
with concurrent.futures.ThreadPoolExecutor() as executor:
    future1 = executor.submit(comfyui)
    future2 = executor.submit(venv_install)
    future3 = executor.submit(model)
libtcmalloc()
start()

采用多线程启动，加快启动速度
-= 线程1：安装ComfyUI -=
-= 线程2：安装apt-get和Python环境 -=
-= 线程3：30秒后自动下载模型 -=
/kaggle/working
-= ready to git colne ComfyUI from QQ2575044704 =-
Cloning into 'ComfyUI'...
Get:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [5002 B]
Get:2 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]       
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]      
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Get:6 https://packages.cloud.google.com/apt google-fast-socket InRelease [5015 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]        
Get:8 http://packages.cloud.google.com/apt gcsfuse-focal/main amd64 Packages [2673 B]
0% [7 InRelease 38.8 kB/119 kB 33%] [4 InRelease 69.2 kB/110 kB 63%]           remote: Enumerating objects: 6008, done.
remote: Counting objects: 100% (302/3

The following additional packages will be installed:
  liblzma-dev
Suggested packages:
  liblzma-doc
The following NEW packages will be installed:
  liblzma-dev libunwind-dev


dpkg-preconfigure: unable to re-open stdin: No such file or directory


0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 2040 kB of archives.
After this operation, 6754 kB of additional disk space will be used.
Selecting previously unselected package liblzma-dev:amd64.
(Reading database ... 113791 files and directories currently installed.)
Preparing to unpack .../liblzma-dev_5.2.5-2ubuntu1_amd64.deb ...
Unpacking liblzma-dev:amd64 (5.2.5-2ubuntu1) ...
Selecting previously unselected package libunwind-dev:amd64.
Preparing to unpack .../libunwind-dev_1.3.2-2build2_amd64.deb ...
Unpacking libunwind-dev:amd64 (1.3.2-2build2) ...
Setting up liblzma-dev:amd64 (5.2.5-2ubuntu1) ...
Setting up libunwind-dev:amd64 (1.3.2-2build2) ...
Processing triggers for man-db (2.10.2-1) ...
Selecting previously unselected package google-perftools.
(Reading database ... 113884 files and directories currently installed.)
Preparing to unpack google-perftools_2.5-2.2ubuntu3_all.deb ...
Unpacking google-perftools (2.5-2.2ubuntu3) ...
Selecting previously 

In [7]:
#部署完毕后快速启动，手动执行，PERSISTENCE设置为Files Only
threading.Thread(target=iframe_thread, daemon=True, args=(8188,)).start()
%cd
%cd /kaggle/working/ComfyUI
!/opt/conda/envs/venv/bin/python3 main.py

/root
/kaggle/working/ComfyUI

ComfyUI finished loading, trying to launch localtunnel (if it gets stuck here localtunnel is having issues)

请通过公网IP进入ComfyUI，你的公网IP地址是: 34.134.167.234 
your url is: https://green-peas-beam.loca.lt
^C
Traceback (most recent call last):
  File "/kaggle/working/ComfyUI/main.py", line 66, in <module>
    import comfy.utils
  File "/kaggle/working/ComfyUI/comfy/utils.py", line 1, in <module>
    import torch
  File "/opt/conda/envs/venv/lib/python3.10/site-packages/torch/__init__.py", line 1465, in <module>
    from . import _meta_registrations
  File "/opt/conda/envs/venv/lib/python3.10/site-packages/torch/_meta_registrations.py", line 7, in <module>
    from torch._decomp import _add_op_to_registry, global_decomposition_table, meta_table
  File "/opt/conda/envs/venv/lib/python3.10/site-packages/torch/_decomp/__init__.py", line 169, in <module>
    import torch._decomp.decompositions
  File "/opt/conda/envs/venv/lib/python3.10/site-packages/torch/_decomp/dec

# 必看的注意事项：
# 跑SDXL需要的工作流：https://huggingface.co/datasets/ACCA225/SDXL1-0/resolve/main/SDXL%201.0%20v0.2%20json
# 你也可以去用一用别人的工作流，不一定要用我的，本人的工作流暂时还不支持Lora(因为不会写)
# 跑refiner可能会造成爆内存，暂时无法解决，等一个优化的SDXL工作流